** 최근 답변순 기준 전문가 프로필 수집 및 참여 게시글 수집 - 의료 분야**

웹 구조 참고: https://kin.naver.com/people/expert/index.naver?type=DOCTOR&edirId=0&orgId=0&sort=answerDate

아래 코드1과 2는 파일럿용으로 1페이지 최근 10명의 전문가 프로필만 수집하기때문에, 그 이상의 전문가 프로필을 수집하기 위해서는 페이지네이션 설정이 필요

#Crawling code 1

각 전문가 프로필 링크에 있는 최근 게시물 5개만 크롤링하는 방식

웹 구조 참고: https://kin.naver.com/userinfo/expert/index.naver?u=GTgLNd5DAJfeHV%2FMKmwkORb2v%2Ffzxs5HDtRs9JgJnJ4%3D

In [5]:
#최근

import requests
from bs4 import BeautifulSoup
import csv
import time

BASE_URL = "https://kin.naver.com"
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/112.0.0.0 Safari/537.36")
}

def get_expert_links(list_url):
    response = requests.get(list_url, headers=HEADERS)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    expert_links = []
    # 예시 선택자 – 전문가 프로필 링크 추출 (필요에 따라 수정)
    for a in soup.select("ul.pro_list li dl dt a"):
        href = a.get("href")
        if href and href.startswith("/userinfo/expert/"):
            expert_links.append(BASE_URL + href)
    return expert_links

def get_profile_posts(profile_url):
    response = requests.get(profile_url, headers=HEADERS)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    post_links = []
    # 예시 선택자 – 프로필 페이지에서 '전문가 답변' 게시글 링크 추출 (필요에 따라 수정)
    for a in soup.select("ul.professional_answer li dl dt a"):
        href = a.get("href")
        if href:
            if not href.startswith("http"):
                href = BASE_URL + href
            post_links.append(href)
    return post_links

def parse_post(post_url):
    """
    개별 게시글 페이지에서 질문 정보와 해당 게시글에 있는 모든 답변을 추출합니다.
    각 답변은 답변 내용, 답변 작성일, 답변자 이름, 그리고 프로필 카드 내 '전문의' 라벨(또는 기타 라벨)을 포함합니다.
    """
    response = requests.get(post_url, headers=HEADERS)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    # --- 질문 정보 추출 ---
    # 질문 제목: 예) div.endTitleSection
    qt_elem = soup.select_one("div.endTitleSection")
    question_title = qt_elem.get_text(separator=" ", strip=True) if qt_elem else ""

    # 질문 내용: 예) div.questionDetail (필요시 클래스 추가)
    qc_elem = soup.select_one("div.questionDetail")
    question_content = qc_elem.get_text(separator="\n", strip=True) if qc_elem else ""

    # 질문자 정보: userInfo 영역
    qi_elem = soup.select_one("div.userInfo.userInfo__bullet")
    if qi_elem:
        # 질문자명 (예: "비공개" 또는 실제 이름)
        questioner_elem = qi_elem.select_one("span.infoHeadItem")
        questioner = questioner_elem.get_text(strip=True) if questioner_elem else ""
        # 조회수 및 작성일
        view_count = ""
        question_created = ""
        info_spans = qi_elem.select("span.infoItem")
        for span in info_spans:
            text = span.get_text(strip=True)
            if "조회수" in text:
                view_count = text.replace("조회수", "").strip()
            elif "작성일" in text:
                question_created = text.replace("작성일", "").strip()
    else:
        questioner = view_count = question_created = ""

    # --- 답변 정보 추출 ---
    answers = []
    # 각 답변 블럭은 보통 div.answerArea._contentWrap._answer 로 구성됨
    for answer_block in soup.select("div.answerArea._contentWrap._answer"):
        # 답변 내용
        answer_elem = answer_block.select_one("div.answerDetail._endContents._endContentsText")
        answer_text = answer_elem.get_text(separator="\n", strip=True) if answer_elem else ""

        # 답변 작성일
        answer_date_elem = answer_block.select_one("div.answerInfo p.answerDate")
        answer_date = answer_date_elem.get_text(strip=True) if answer_date_elem else ""

        # 답변자 정보 추출 (프로필 카드 내 정보)
        profile_card = answer_block.select_one("div.profile_card._profileCardArea")
        if profile_card:
            name_elem = profile_card.select_one("strong.name")
            answerer_name = name_elem.get_text(strip=True) if name_elem else ""
            # 전문가 라벨 추출 (예: span.badge.expert_job 또는 다른 배지)
            badge_elem = profile_card.select_one("div.badge_area span.badge")
            expert_label = badge_elem.get_text(strip=True) if badge_elem else ""
        else:
            answerer_name = ""
            expert_label = ""

        answers.append({
            "answer_text": answer_text,
            "answer_date": answer_date,
            "answerer_name": answerer_name,
            "expert_label": expert_label
        })

    return {
        "post_url": post_url,
        "question_title": question_title,
        "question_content": question_content,
        "questioner": questioner,
        "view_count": view_count,
        "question_created": question_created,
        "answers": answers
    }

def main():
    # 전문가 목록 페이지 URL (예시)
    list_url = "https://kin.naver.com/people/expert/index.naver?type=DOCTOR"
    print("전문가 프로필 링크 추출 중...")
    expert_links = get_expert_links(list_url)
    print(f"총 {len(expert_links)}개의 전문가 프로필 링크가 추출되었습니다.")

    # 시험용으로 10명의 전문가만 처리 (필요에 따라 제거)
    expert_links = expert_links[:10]

    dataset = []
    for expert_url in expert_links:
        print("\n-------------------------------------")
        print("전문가 프로필 페이지:", expert_url)
        try:
            post_links = get_profile_posts(expert_url)
            print(f"해당 프로필에서 {len(post_links)}개의 게시글 링크 추출됨")
            for post_url in post_links:
                print("게시글 URL:", post_url)
                try:
                    data = parse_post(post_url)
                    # 어떤 전문가의 게시글인지 추가 정보로 프로필 URL 저장
                    data["expert_profile"] = expert_url
                    dataset.append(data)
                except Exception as e:
                    print("  게시글 파싱 중 오류 발생:", e)
                time.sleep(1)
        except Exception as e:
            print("프로필 페이지 크롤링 중 오류 발생:", e)
        time.sleep(2)

    if dataset:
        output_file = "temp_jisikin_recent_5_medical_250203.csv"
        fieldnames = [
            "post_url", "expert_profile",
            "question_title", "question_content",
            "questioner", "view_count", "question_created",
            "answer_text", "answer_date", "answerer_name", "expert_label"
        ]
        with open(output_file, "w", encoding="utf-8-sig", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for data in dataset:
                # 한 게시글에 여러 답변이 있는 경우 각 답변마다 한 행(row)으로 저장
                for answer in data["answers"]:
                    row = {
                        "post_url": data["post_url"],
                        "expert_profile": data.get("expert_profile", ""),
                        "question_title": data["question_title"],
                        "question_content": data["question_content"],
                        "questioner": data["questioner"],
                        "view_count": data["view_count"],
                        "question_created": data["question_created"],
                        "answer_text": answer["answer_text"],
                        "answer_date": answer["answer_date"],
                        "answerer_name": answer["answerer_name"],
                        "expert_label": answer["expert_label"]
                    }
                    writer.writerow(row)
        print(f"\n데이터셋이 '{output_file}' 파일에 저장되었습니다.")
    else:
        print("추출된 데이터가 없습니다.")

if __name__ == "__main__":
    main()


전문가 프로필 링크 추출 중...
총 10개의 전문가 프로필 링크가 추출되었습니다.

-------------------------------------
전문가 프로필 페이지: https://kin.naver.com/userinfo/expert/index.naver?u=GTgLNd5DAJfeHV%2FMKmwkORb2v%2Ffzxs5HDtRs9JgJnJ4%3D
해당 프로필에서 5개의 게시글 링크 추출됨
게시글 URL: https://kin.naver.com/qna/detail.naver?d1id=7&dirId=70110&docId=481273885&answerNo=2


KeyboardInterrupt: 

#Crawling code 2
전문가 프로필 링크에 더보기 항목을 통해서 추가적인 게시글 url 가져오는 기능 추가, 전체를 다 가져올 수 있지만, 수집량이 너무 많기 때문에 한명의 전문가 프로필 1페이지내 10개만 제한함.

웹 구조 참고: https://kin.naver.com/userinfo/expert/index.naver?u=GTgLNd5DAJfeHV%2FMKmwkORb2v%2Ffzxs5HDtRs9JgJnJ4%3D

In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time
from urllib.parse import urlparse, parse_qs

BASE_URL = "https://kin.naver.com"
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/112.0.0.0 Safari/537.36")
}

def get_expert_links(list_url):
    response = requests.get(list_url, headers=HEADERS)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    expert_links = []
    # 전문가 프로필 링크 추출 (실제 HTML 구조에 따라 선택자 수정)
    for a in soup.select("ul.pro_list li dl dt a"):
        href = a.get("href")
        if href and href.startswith("/userinfo/expert/"):
            expert_links.append(BASE_URL + href)
    return expert_links

def get_profile_posts(profile_url):
    """
    전문가 프로필 페이지에서 최근 목록에 표시된 게시글 URL들을 추출합니다.
    """
    response = requests.get(profile_url, headers=HEADERS)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    post_links = []
    # 전문가 프로필 내 최근 게시글 목록 선택자 (필요에 따라 수정)
    for a in soup.select("ul.professional_answer li dl dt a"):
        href = a.get("href")
        if href:
            if not href.startswith("http"):
                href = BASE_URL + href
            post_links.append(href)
    return post_links

def get_all_expert_posts(more_url, limit=10):
    """
    전문가 프로필 '더보기' 페이지에서 전체 게시글 목록 중 최대 limit개(기본 10개) 게시글 URL을 추출합니다.
    more_url은 예를 들어 "https://kin.naver.com/userinfo/expert/answerList.naver?u=인코딩된ID"와 같이 시작됩니다.
    """
    all_post_links = []
    page = 1
    while True:
        url = f"{more_url}&page={page}"
        print(f"    [더보기] 요청 페이지: {url}")
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        # 예시 선택자 – 테이블 내 게시글 목록의 링크 추출 (필요에 따라 수정)
        rows = soup.select("table.boardtype2.th_border tbody#au_board_list tr")
        if not rows:
            break  # 더 이상 게시글이 없으면 종료
        for row in rows:
            a = row.select_one("td.title a")
            if a:
                href = a.get("href")
                if href and not href.startswith("http"):
                    href = BASE_URL + href
                all_post_links.append(href)
                if len(all_post_links) >= limit:
                    break
        if len(all_post_links) >= limit:
            break
        # "다음" 버튼이 없으면 종료
        next_link = soup.select_one("div.paginate a.next")
        if next_link is None:
            break
        page += 1
        time.sleep(1)
    return all_post_links[:limit]

def extract_expert_id(profile_url):
    """
    전문가 프로필 URL에서 인코딩된 'u' 파라미터 값을 추출합니다.
    예: "https://kin.naver.com/profile/index.naver?u=mrSqA%2BuSw%2FdLfiyYPQ1RiHTd%2BSaDuLES6f1qnbJKBZU%3D"
    """
    parsed = urlparse(profile_url)
    qs = parse_qs(parsed.query)
    return qs.get('u', [None])[0]

def parse_post(post_url):
    """
    개별 게시글 페이지에서 질문 제목, 질문 내용, 질문자 정보와
    각 답변의 내용, 작성일, 작성자 및 전문가(또는 기타) 배지 정보를 추출합니다.
    """
    response = requests.get(post_url, headers=HEADERS)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    # --- 질문 정보 추출 ---
    qt_elem = soup.select_one("div.endTitleSection")
    question_title = qt_elem.get_text(separator=" ", strip=True) if qt_elem else ""

    qc_elem = soup.select_one("div.questionDetail")
    question_content = qc_elem.get_text(separator="\n", strip=True) if qc_elem else ""

    qi_elem = soup.select_one("div.userInfo.userInfo__bullet")
    if qi_elem:
        questioner_elem = qi_elem.select_one("span.infoHeadItem")
        questioner = questioner_elem.get_text(strip=True) if questioner_elem else ""
        view_count = ""
        question_created = ""
        info_spans = qi_elem.select("span.infoItem")
        for span in info_spans:
            text = span.get_text(strip=True)
            if "조회수" in text:
                view_count = text.replace("조회수", "").strip()
            elif "작성일" in text:
                question_created = text.replace("작성일", "").strip()
    else:
        questioner = view_count = question_created = ""

    # --- 답변 정보 추출 ---
    answers = []
    # 답변 블럭 선택자 (실제 HTML에 맞게 수정)
    for answer_block in soup.select("div.answerArea._contentWrap._answer"):
        answer_elem = answer_block.select_one("div.answerDetail._endContents._endContentsText")
        answer_text = answer_elem.get_text(separator="\n", strip=True) if answer_elem else ""

        answer_date_elem = answer_block.select_one("div.answerInfo p.answerDate")
        answer_date = answer_date_elem.get_text(strip=True) if answer_date_elem else ""

        # 프로필 카드 내 정보
        profile_card = answer_block.select_one("div.profile_card._profileCardArea")
        if profile_card:
            name_elem = profile_card.select_one("strong.name")
            answerer_name = name_elem.get_text(strip=True) if name_elem else ""
            badge_elem = profile_card.select_one("div.badge_area span.badge")
            expert_label = badge_elem.get_text(strip=True) if badge_elem else ""
        else:
            answerer_name = ""
            expert_label = ""

        answers.append({
            "answer_text": answer_text,
            "answer_date": answer_date,
            "answerer_name": answerer_name,
            "expert_label": expert_label
        })

    return {
        "post_url": post_url,
        "question_title": question_title,
        "question_content": question_content,
        "questioner": questioner,
        "view_count": view_count,
        "question_created": question_created,
        "answers": answers
    }

def main():
    # 전문가 목록 페이지 URL (예시)
    list_url = "https://kin.naver.com/people/expert/index.naver?type=DOCTOR"
    print("전문가 프로필 링크 추출 중...")
    expert_links = get_expert_links(list_url)
    print(f"총 {len(expert_links)}개의 전문가 프로필 링크가 추출되었습니다.")

    # 예제에서는 테스트용으로 10명의 전문가만 처리 (필요에 따라 조정)
    expert_links = expert_links[:10]

    dataset = []
    for expert_url in expert_links:
        print("\n-------------------------------------")
        print("전문가 프로필 페이지:", expert_url)
        try:
            # ① 프로필 페이지에서 최근 게시글 목록 추출
            recent_post_links = get_profile_posts(expert_url)
            print(f"  [최근 게시글] {len(recent_post_links)}개 링크 추출됨")
            # ② "더보기" 페이지에서 최대 10개의 게시글 URL 추출
            encoded_id = extract_expert_id(expert_url)
            if encoded_id:
                more_url = f"https://kin.naver.com/userinfo/expert/answerList.naver?u={encoded_id}"
                additional_post_links = get_all_expert_posts(more_url, limit=10)
                print(f"  [더보기 게시글] {len(additional_post_links)}개 링크 추출됨")
            else:
                additional_post_links = []

            # 중복 제거
            all_post_links = list(set(recent_post_links + additional_post_links))
            print(f"  총 {len(all_post_links)}개의 게시글 URL 확보됨")
            for post_url in all_post_links:
                print("    게시글 URL:", post_url)
                try:
                    data = parse_post(post_url)
                    data["expert_profile"] = expert_url
                    dataset.append(data)
                except Exception as e:
                    print("      게시글 파싱 중 오류 발생:", e)
                time.sleep(1)
        except Exception as e:
            print("  전문가 프로필 크롤링 중 오류 발생:", e)
        time.sleep(2)

    if dataset:
        output_file = "dataset.csv"
        fieldnames = [
            "post_url", "expert_profile",
            "question_title", "question_content",
            "questioner", "view_count", "question_created",
            "answer_text", "answer_date", "answerer_name", "expert_label"
        ]
        with open(output_file, "w", encoding="utf-8-sig", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for data in dataset:
                for answer in data["answers"]:
                    row = {
                        "post_url": data["post_url"],
                        "expert_profile": data.get("expert_profile", ""),
                        "question_title": data["question_title"],
                        "question_content": data["question_content"],
                        "questioner": data["questioner"],
                        "view_count": data["view_count"],
                        "question_created": data["question_created"],
                        "answer_text": answer["answer_text"],
                        "answer_date": answer["answer_date"],
                        "answerer_name": answer["answerer_name"],
                        "expert_label": answer["expert_label"]
                    }
                    writer.writerow(row)
        print(f"\n데이터셋이 '{output_file}' 파일에 저장되었습니다.")
    else:
        print("추출된 데이터가 없습니다.")

if __name__ == "__main__":
    main()


전문가 프로필 링크 추출 중...
총 10개의 전문가 프로필 링크가 추출되었습니다.

-------------------------------------
전문가 프로필 페이지: https://kin.naver.com/userinfo/expert/index.naver?u=GTgLNd5DAJfeHV%2FMKmwkORb2v%2Ffzxs5HDtRs9JgJnJ4%3D
  [최근 게시글] 5개 링크 추출됨
    [더보기] 요청 페이지: https://kin.naver.com/userinfo/expert/answerList.naver?u=GTgLNd5DAJfeHV/MKmwkORb2v/fzxs5HDtRs9JgJnJ4=&page=1
  [더보기 게시글] 10개 링크 추출됨
  총 10개의 게시글 URL 확보됨
    게시글 URL: https://kin.naver.com/qna/detail.naver?d1id=7&dirId=70110&docId=481210874&answerNo=2
    게시글 URL: https://kin.naver.com/qna/detail.naver?d1id=7&dirId=70110&docId=481186642&answerNo=1
    게시글 URL: https://kin.naver.com/qna/detail.naver?d1id=7&dirId=70110&docId=481232745&answerNo=1
    게시글 URL: https://kin.naver.com/qna/detail.naver?d1id=7&dirId=70110&docId=481198060&answerNo=1
    게시글 URL: https://kin.naver.com/qna/detail.naver?d1id=7&dirId=70110&docId=481206711&answerNo=2
    게시글 URL: https://kin.naver.com/qna/detail.naver?d1id=7&dirId=70110&docId=481152716&answerNo=1
    게시글 URL: https